https://divamgupta.com/image-segmentation/2019/06/06/deep-learning-semantic-segmentation-keras.html Refer to this for reasource


TODO: Add optimisers

In [1]:
!pip install keras-segmentation
#https://github.com/divamgupta/image-segmentation-keras


  Created wheel for keras-segmentation: filename=keras_segmentation-0.3.0-py3-none-any.whl size=29071 sha256=4d0f9d45f81a55d1baed9d20b1d3b589eefd1971e0183dc5132fa2f1a429fba3
  Stored in directory: c:\users\seanh\appdata\local\pip\cache\wheels\8f\86\73\3dc66376905add0f57142bdd6e2117db5d4aa6340e34d039c7
Successfully built keras-segmentation
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0


In [ ]:
#make rgb images and segmentation images share a file name
#get image in the right format

import os
from numpy import asarray
from PIL import Image

rgb_img = r'/home/test_images'
seg_img = r'/home/test_segmentation/'
rename = "test_"
i = 0

def remove_chars(filename):
  new_filename = ""
  for i in range(0, len(filename)):
    char = filename[i]
    if char.isnumeric():
      new_filename += char
  return new_filename + ".png"

for filename in os.listdir(rgb_img):
    old_path = os.path.join(rgb_img, filename)
    if not os.path.isdir(old_path):
      i += 1
      new_filename = remove_chars(filename)
      os.rename(os.path.join(rgb_img, filename), os.path.join(rgb_img, new_filename))

i = 0
for filename in os.listdir(seg_img):
    old_path = os.path.join(seg_img, filename)
    if not os.path.isdir(old_path):
      i += 1
      new_filename = remove_chars(filename)
      new_name = os.path.join(seg_img,new_filename)
      os.rename(os.path.join(seg_img, filename), os.path.join(seg_img, new_name))
      
      
      #Normalize image to fit model system: https://machinelearningmastery.com/how-to-manually-scale-image-pixel-data-for-deep-learning/ 
      image = Image.open(new_name)
      pixels = asarray(image)
      # convert from integers to floats
      pixels = pixels.astype('float32')
      # normalize to the range 0-1 then to 9
      pixels /= 255.0
      pixels *= 9.0
      print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))
      im = Image.fromarray(pixels.astype(np.uint8))
      im.save() 

FileNotFoundError: ignored

In [ ]:
!python -m keras_segmentation verify_dataset \
 --images_path="/home/test_images" \
 --segs_path="/home/test_segmentation/"  \
 --n_classes=10

Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/keras_segmentation/__main__.py", line 7, in <module>
    main()
  File "/usr/local/lib/python3.7/dist-packages/keras_segmentation/__main__.py", line 4, in main
    cli_interface.main()
  File "/usr/local/lib/python3.7/dist-packages/keras_segmentation/cli_interface.py", line 142, in main
    args.func(args)
  File "/usr/local/lib/python3.7/dist-packages/keras_segmentation/cli_interface.py", line 105, in action
    args.images_path, args.segs_path, args.n_classes)
  File "/usr/local/lib/python3.7/dist-packages/keras_segmentation/data_utils/data_loader.py", line 150, in verify_segmentation_dataset
    img_seg_pairs = get_pairs_from_paths(images_path, segs_path)
  File "/usr/local/lib/python3.7/dist-packages/keras_segmentation/

In [ ]:
import cv2
import numpy as np
from keras_segmentation import *

ModuleNotFoundError: ignored

Create numpy dataset for data

In [ ]:
from keras.models import Model, model_from_json
from keras.layers import Input, Dense, Conv2D, Dropout, MaxPooling2D, concatenate, UpSampling2D
from keras.callbacks import ModelCheckpoint
import keras
from tensorflow import keras
n_classes = 10
input_height = 1080
input_width = 720
img_input = Input(shape=(input_height,input_width , 3 ))

conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(img_input)
conv1 = Dropout(0.2)(conv1)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Dropout(0.2)(conv2)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D((2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Dropout(0.2)(conv3)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

up1 = concatenate([UpSampling2D((2, 2))(conv3), conv2], axis=-1)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
conv4 = Dropout(0.2)(conv4)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

up2 = concatenate([UpSampling2D((2, 2))(conv4), conv1], axis=-1)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
conv5 = Dropout(0.2)(conv5)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)

out = Conv2D( n_classes, (1, 1) , padding='same')(conv5)

from keras_segmentation.models.model_utils import get_segmentation_model


model = get_segmentation_model(img_input ,  out,) # this would build the segmentation model
opt = keras.optimizers.Relu(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt)

ModuleNotFoundError: ignored

In [ ]:
model.train( 
    train_images =  "/home/test_images/",
    train_annotations = "/home/test_segmentation/",
    checkpoints_path = "/home/weights/" , epochs=5
)

Verifying training dataset


100%|██████████| 60/60 [00:01<00:00, 34.27it/s]
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Dataset verified! 
Epoch 1/5
371/512 [====================>.........] - ETA: 1:24:48 - loss: 3.9610 - accuracy: 0.3736